In [ ]:
!pip install https://github.com/facebookresearch/fastText/archive/master.zip

In [1]:
from deeppavlov.models.classifiers.keras_cnn_classifier import keras_cnn_classifier
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


/home/nab/work/deepp/venv-pilot/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using TensorFlow backend.


In [2]:
categories = [
    "alt.atheism",
    "sci.space"
]
train_data = fetch_20newsgroups(subset='train', categories=categories)
test_data = fetch_20newsgroups(subset='test', categories=categories)

In [31]:
print(train_data.target[130])
print(len(train_data.data))
print(len(train_data.target_names))
print(train_data.target_names)
print()
print(train_data.data[144])


1
1073
2
['alt.atheism', 'sci.space']

From: bil@okcforum.osrhe.edu (Bill Conner)
Subject: Re: Death Penalty (was Re: Political Atheists?)
Nntp-Posting-Host: okcforum.osrhe.edu
Organization: Okcforum Unix Users Group
X-Newsreader: TIN [version 1.1 PL9]
Lines: 14

This is fascinating. Atheists argue for abortion, defend homosexuality
as a means of population control, insist that the only values are
biological and condemn war and capital punishment. According to
Benedikt, if something is contardictory, it cannot exist, which in
this case means atheists I suppose.
I would like to understand how an atheist can object to war (an
excellent means of controlling population growth), or to capital
punishment, I'm sorry but the logic escapes me.
And why just capital punishment, what is being questioned here, the
propriety of killing or of punishment? What is the basis of the
ecomplaint?

Bill




In [4]:
import fastText
embedder = fastText.load_model('DeepPavlov/download/embeddings/wiki.en.bin')

In [5]:
train_vectors = [np.sum([embedder.get_sentence_vector(line) for line in x.split()], axis=0) for x in train_data.data]

In [6]:
mlp_model = MLPClassifier()


In [7]:
mlp_model.fit(train_vectors, train_data.target)

In [13]:
mlp_model.predict(embedder.get_sentence_vector("scientific text about space").reshape(1, 300))

In [32]:
keras_params = {"n_classes": 1,  # 1 for binary
                "text_size": 1,
                "embedding_size": len(train_vectors[0])}
sk_params = {
    # "batch_size": 16, "epochs": 3}
}
my_model = KerasClassifier(build_fn=keras_cnn_classifier, build_fn_params=keras_params, **sk_params)


In [33]:
my_model.fit(np.array(train_vectors)[:, np.newaxis], train_data.target)

Epoch 1/1


  32/1073 [..............................] - ETA: 36s - loss: 0.8571 - binary_accuracy: 0.5938 - fmeasure: 0.6486

 288/1073 [=======>......................] - ETA: 3s - loss: 0.6975 - binary_accuracy: 0.6771 - fmeasure: 0.6760 

 544/1073 [==============>...............] - ETA: 1s - loss: 0.6677 - binary_accuracy: 0.7114 - fmeasure: 0.7273

 832/1073 [======================>.......] - ETA: 0s - loss: 0.6487 - binary_accuracy: 0.7344 - fmeasure: 0.7557

1073/1073 [==============================] - 1s 1ms/step - loss: 0.6426 - binary_accuracy: 0.7381 - fmeasure: 0.7588


In [ ]:
pipeline = Pipeline([
    ('emb', FastTextEmbedder()),
    ('clf', my_model),
])
